# HTTP Request/Response Cycle - Lab

## Introduction 

In this lab, we'll make use of the `requests` module commands and properties seen in the previous lesson, to extract information for a web service called **"Open Notify"** to access NASA's space data. 

## Objectives

You will be able to:

* Explain the HTTP request/response cycle
* List the status codes of responses and their meanings
* Obtain and interpret status codes from responses
* Make HTTP GET and POST requests in python using the `requests` library

## Open Notify 

[Open Notify](http://open-notify.org/)  is an an open source project to provide a simple programming interface for some of NASA’s awesome data. This takes live raw data from NASA's systems and turn them into APIs related to space and spacecraft. We can access the following information from open notify. 

* Current Location of the International Space Station

* Overhead Pass Predictions for the International Space Station

* Number of People in Space
    
### API endpoints

OpenNotify has several API endpoints. 
>An endpoint is a server route that is used to retrieve different data from the API. 

For example, the `/comments` endpoint on the Reddit API might retrieve information about comments, whereas the `/users` endpoint might retrieve data about users. To access them, you would add the endpoint to the base url of the API.

For the OpenNotify API, we have following end points 

1. Current Location of the International Space Station `/iss-now.json`
2. Overhead Pass Predictions for the International Space Station `/iss-pass.json`    
3. Number of People in Space `/astros.json`

The `.json` extension simply tells us that the data is being returned in a JSON format.

In this lab, we'll be querying this API to retrieve live data about the International Space Station (ISS). Details on OpenNofity, endpoints, syntax, and the services it offers can be viewed [Here](http://open-notify.org/Open-Notify-API/)

![](images/iss.jpg)

### Current location of International Space Station

The first endpoint we'll look at on OpenNotify is the ` iss-now.json` endpoint (current location of international space station). This endpoint gets the current latitude and longitude of the International Space Station.  Perform the following tasks 
* Make a get request to get the latest position of the international space station from the opennotify api's `iss-now` endpoint at http://api.open-notify.org/iss-now.json
* Check the status code of the response
* Interpret the returned code

In [1]:
# Your Code Here
import requests

resp = requests.get('http://api.open-notify.org/iss-now.json')
print(f"resp.status_code=={resp.status_code} == 200? {resp.status_code == requests.codes.ok}")

resp.status_code==200 == 200? True


HTTP respode is 200, so the request was successful.

* Print the contents of the response and identify its current location

In [2]:
# You Code Here
print(resp.text)

{"iss_position": {"longitude": "-74.2556", "latitude": "7.1824"}, "timestamp": 1579980783, "message": "success"}


In [3]:
# Interpret your results using the API
response = resp.json()
print(f"""Location of the International Space Station at timestamp {response['timestamp']} is: 
    LONG={response['iss_position']['longitude']}, 
    LAT={response['iss_position']['latitude']}
""")

Location of the International Space Station at timestamp 1579980783 is: 
    LONG=-74.2556, 
    LAT=7.1824



### Check the next pass of International space station for a given location

Let's repeat the above for the second endpoint `iss-pass.json`. This end point is used to query the next pass of the space station on a given location. Let's just run as above and record your observations.

In [4]:
# You Code Here
resp = requests.get('http://api.open-notify.org/iss-pass.json')
print(f"resp.status_code=={resp.status_code} == 200? {resp.status_code == requests.codes.ok}")
print(resp.text)
response = resp.json()
print(f"""Next pass of the International Space Station will be: 
    LONG={response['iss_position']['longitude']}, 
    LAT={response['iss_position']['latitude']}
""")

resp.status_code==400 == 200? False
{
  "message": "failure", 
  "reason": "Latitude must be specified"
}



KeyError: 'iss_position'

The API HTTP request fails with response code 400.

So clearly there is something wrong as we had a 400 response. This is how you should always test your responses for validity. 

if we look at the documentation for the OpenNotify API, we see that the ISS Pass endpoint requires two parameters.

> The ISS Pass endpoint returns when the ISS will next pass over a given location on earth. In order to compute this, we need to pass the coordinates of the location to the API. We do this by passing two parameters -- latitude and longitude.

We can do this by adding an optional keyword argument, params, to our request. In this case, there are two parameters we need to pass:

* lat -- The latitude of the location we want.
* lon -- The longitude of the location we want.

Perform the following tasks :
* Set parameters to reflect the lat and long of New York  (40.71, -74)
* Send a get request to OpenNotify passing in the lat long parameters as k:v pairs in a dictionary
* Check the status code and interpret
* Print the header information and the returned content

In [5]:
# You Code Here
location = {'lat': 40.71, 'lon': -74}  
r = requests.get('http://api.open-notify.org/iss-pass.json', params=location)
print(f"resp.status_code=={r.status_code} == 200? {r.status_code == requests.codes.ok}")
d_headers = None
if r.status_code==200:
    d_headers = dict(resp.headers)
    print(d_headers, "\n")
    print(r.text)
    response = r.json()

resp.status_code==200 == 200? True
{'Server': 'nginx/1.10.3', 'Date': 'Sat, 25 Jan 2020 19:33:04 GMT', 'Content-Type': 'application/json', 'Content-Length': '70', 'Connection': 'keep-alive', 'Via': '1.1 vegur'} 

{
  "message": "success", 
  "request": {
    "altitude": 100, 
    "datetime": 1579978162, 
    "latitude": 40.71, 
    "longitude": -74.0, 
    "passes": 5
  }, 
  "response": [
    {
      "duration": 255, 
      "risetime": 1579981105
    }, 
    {
      "duration": 636, 
      "risetime": 1579986672
    }, 
    {
      "duration": 638, 
      "risetime": 1579992480
    }, 
    {
      "duration": 569, 
      "risetime": 1579998371
    }, 
    {
      "duration": 586, 
      "risetime": 1580004234
    }
  ]
}



In [6]:
# Check the API and interpret your results - when will ISS pass over NEW York next ?
if r.status_code==200:
    passes = response['response']
    print(f"{len(passes)} passes reported for next pass.")
    print(f"The ISS will next pass over NY at location ({location['lat']}, {location['lon']}) at risetime {passes[0]['risetime']} for a duration of {passes[0]['duration']} (seconds?)")

5 passes reported for next pass.
The ISS will next pass over NY at location (40.71, -74) at risetime 1579981105 for a duration of 255 (seconds?)


### Finding the number of people in space

OpenNotify has one more API endpoint, `/astros.json`. It tells you how many people are currently in space. The format of the responses can be studied [HERE](http://open-notify.org/Open-Notify-API/People-In-Space/).

Read the above documentation and perform the following tasks:

* Get the response from astros.json endpoint
* Count how many people are currently in space
* List the names of people currently in space.

In [7]:
# You Code Here
r = requests.get('http://api.open-notify.org/astros.json')
print(f"resp.status_code=={r.status_code} == 200? {r.status_code == requests.codes.ok}")
d_headers = None
if r.status_code==200:
    print(r.text)
    response = r.json()

resp.status_code==200 == 200? True
{"people": [{"name": "Christina Koch", "craft": "ISS"}, {"name": "Alexander Skvortsov", "craft": "ISS"}, {"name": "Luca Parmitano", "craft": "ISS"}, {"name": "Andrew Morgan", "craft": "ISS"}, {"name": "Oleg Skripochka", "craft": "ISS"}, {"name": "Jessica Meir", "craft": "ISS"}], "number": 6, "message": "success"}


In [8]:
# Interpret the Results - How many people are in space and what are their names 
if r.status_code==200:
    people = response['people']
    print("The following {len(people)} astronauts are in space:")
    for person in people:
        print(f"\t{person['name']}, on the {person['craft']}")

The following {len(people)} astronauts are in space:
	Christina Koch, on the ISS
	Alexander Skvortsov, on the ISS
	Luca Parmitano, on the ISS
	Andrew Morgan, on the ISS
	Oleg Skripochka, on the ISS
	Jessica Meir, on the ISS


## Summary 

In this lesson, we saw how we can use request and response methods to query an Open API. We also saw how to look at the contents returned with the API calls and how to parse them. Next, we'll look at connecting to APIs which are not OPEN, i.e. we would need to pass in some authentication information and filter the results. 